[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uwsampl/tutorial/blob/master/notebook/04b_TVM_Tutorial_VTA_ResNet.ipynb)

Please run the following block to ensure TVM is setup for this notebook, each notebook may have its own runtime.

In [1]:
%matplotlib inline

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    ! gsutil cp "gs://tvm-fcrc-binaries-7f775516ff9dfab922c304049f294cec/tvm.tar.gz" /tmp/tvm.tar.gz
    ! mkdir -p /tvm
    ! tar -xf /tmp/tvm.tar.gz --strip-components=4 --directory /tvm
    ! ls -la /tvm
    ! pip install mxnet
    ! bash /tvm/package.sh
    # Add TVM to the Python path.
    import sys
    sys.path.append('/tvm/python')
    sys.path.append('/tvm/topi/python')
    sys.path.append('/tvm/nnvm/python')
    sys.path.append('/tvm/vta/python')
else:
    print("Notebook executing locally, skipping Colab setup ...")

Notebook executing locally, skipping Colab setup ...



Deploy Pretrained ResNet Model from MxNet on VTA
================================================
**Author**: [Thierry Moreau](https://homes.cs.washington.edu/~moreau/)

This tutorial provides an end-to-end demo, on how to run ResNet-18 inference
onto the VTA accelerator design to perform ImageNet classification tasks.
It showcases Relay as a front end compiler that can perform quantization (VTA only supports int8/32 inference) as well as graph packing (in order to enable tensorization in the core) to massage the compute graph for the hardware target.


Import packages
--------------------

In [2]:
from __future__ import absolute_import, print_function

import argparse, json, os, requests, time
from io import BytesIO
from os.path import join, isfile
from PIL import Image

from mxnet.gluon.model_zoo import vision
import numpy as np
from matplotlib import pyplot as plt

import tvm
from tvm import rpc, autotvm, relay
from tvm.contrib import graph_runtime, util, download
from tvm.contrib.debugger import debug_runtime

import vta
from vta.testing import simulator
from vta.top import graph_pack

# Make sure that TVM was compiled with RPC=1
assert tvm.module.enabled("rpc")

Define the platform and model targets
-------------------------------------
Execute on CPU vs. VTA, and define the model.



In [3]:
# Load VTA parameters from the vta/config/vta_config.json file
env = vta.get_env()
print(env.TARGET)

# Set device=arm_cpu to run inference on the CPU
# or device=vta to run inference on the FPGA.
device = "vta"
target = env.target if device == "vta" else env.target_vta_cpu

# Name of Gluon model to compile
model = "resnet18_v1"

sim


Obtain an execution remote
---------------------------------
When target is `pynq`, reconfigure FPGA and runtime.
Otherwise, if target is `sim`, execute locally.



In [4]:
if env.TARGET != "sim":
    
    # Get remote to board
    device_host = os.environ.get("VTA_PYNQ_RPC_HOST", "192.168.2.99")
    device_port = int(os.environ.get("VTA_PYNQ_RPC_PORT", "9091"))
    remote = rpc.connect(device_host, device_port)

    # Reconfigure the JIT runtime and FPGA.
    # You can program the FPGA with your own custom bitstream
    # by passing the path to the bitstream file instead of None.
    vta.reconfig_runtime(remote)
    vta.program_fpga(remote, bitstream=None)

# In simulation mode, host the RPC server locally.
else:
    remote = rpc.LocalSession()

# Get execution context from remote
ctx = remote.ext_dev(0) if device == "vta" else remote.cpu(0)

Build the inference graph runtime
---------------------------------
Grab ResNet-18 model from Gluon model zoo and compile with Relay.
The compilation steps are:
   1. Front end translation from MxNet into Relay module.
   2. Apply 8-bit quantization: here we skip the first conv layer,
      and dense layer which will both be executed in fp32 on the CPU.
   3. Perform graph packing to alter the data layout for tensorization.
   4. Perform constant folding to reduce number of operators (e.g. eliminate batch norm multiply).
   5. Perform relay build to object file.
   6. Load the object file onto remote (FPGA device).
   7. Generate graph runtime, `m`.


In [6]:
# Populate the shape and data type dictionary for ResNet input
dtype_dict = {"data": 'float32'}
shape_dict = {"data": (env.BATCH, 3, 224, 224)}

# Get off the shelf gluon model, and convert to relay
gluon_model = vision.get_model(model, pretrained=True)

# Start front end compilation
relay_prog, params = relay.frontend.from_mxnet(gluon_model, shape_dict)

# Update shape and type dictionary
shape_dict.update({k: v.shape for k, v in params.items()})
dtype_dict.update({k: str(v.dtype) for k, v in params.items()})

# Print the off the shelf model
print(relay_prog)

v0.0.1
fn (%data: Tensor[(1, 3, 224, 224), float32], %resnetv11_conv0_weight: Tensor[(64, 3, 7, 7), float32], %resnetv11_batchnorm0_gamma: Tensor[(64,), float32], %resnetv11_batchnorm0_beta: Tensor[(64,), float32], %resnetv11_batchnorm0_running_mean: Tensor[(64,), float32], %resnetv11_batchnorm0_running_var: Tensor[(64,), float32], %resnetv11_stage1_conv0_weight: Tensor[(64, 64, 3, 3), float32], %resnetv11_stage1_batchnorm0_gamma: Tensor[(64,), float32], %resnetv11_stage1_batchnorm0_beta: Tensor[(64,), float32], %resnetv11_stage1_batchnorm0_running_mean: Tensor[(64,), float32], %resnetv11_stage1_batchnorm0_running_var: Tensor[(64,), float32], %resnetv11_stage1_conv1_weight: Tensor[(64, 64, 3, 3), float32], %resnetv11_stage1_batchnorm1_gamma: Tensor[(64,), float32], %resnetv11_stage1_batchnorm1_beta: Tensor[(64,), float32], %resnetv11_stage1_batchnorm1_running_mean: Tensor[(64,), float32], %resnetv11_stage1_batchnorm1_running_var: Tensor[(64,), float32], %resnetv11_stage1_conv2_weight: 

v0.0.1
fn (%data: Tensor[(1, 3, 224, 224), float32]) -> Tensor[(1, 1000), float32] {
  %0 = nn.conv2d(%data, meta[relay.Constant][0] /* ty=Tensor[(64, 3, 7, 7), float32] */ /* ty=Tensor[(64, 3, 7, 7), float32] */, strides=[2, 2], padding=[3, 3], channels=64, kernel_size=[7, 7]) /* ty=Tensor[(1, 64, 112, 112), float32] */
  %1 = add(%0, meta[relay.Constant][1] /* ty=Tensor[(64, 1, 1), float32] */ /* ty=Tensor[(64, 1, 1), float32] */) /* ty=Tensor[(1, 64, 112, 112), float32] */
  %2 = nn.relu(%1) /* ty=Tensor[(1, 64, 112, 112), float32] */
  %3 = nn.max_pool2d(%2, pool_size=[3, 3], strides=[2, 2], padding=[1, 1]) /* ty=Tensor[(1, 64, 56, 56), float32] */
  %4 = reshape(%3, newshape=[1, 1, 4, 16, 56, 56]) /* ty=Tensor[(1, 1, 4, 16, 56, 56), float32] */
  %5 = transpose(%4, axes=[0, 2, 4, 5, 1, 3]) /* ty=Tensor[(1, 4, 56, 56, 1, 16), float32] */
  %6 = force_cast(%5) /* ty=Tensor[(1, 4, 56, 56, 1, 16), float32] */
  %7 = stop_fusion(%6) /* ty=Tensor[(1, 4, 56, 56, 1, 16), float32] */
  %8 

In [ ]:
# Perform quantization in Relay
with relay.quantize.qconfig(global_scale=8.0,
                            skip_k_conv=1,
                            skip_k_dense=1,
                            target_vta=True):
    relay_prog = relay.quantize.quantize(relay_prog, params=params)

# Perform graph packing and constant folding for VTA target
if target.device_name == "vta":
    assert env.BLOCK_IN == env.BLOCK_OUT
    relay_prog = graph_pack(
        relay_prog,
        env.BATCH,
        env.BLOCK_OUT,
        env.WGT_WIDTH,
        start_name="nn.max_pool2d",
        stop_name="nn.global_avg_pool2d")
    relay_prog = relay.ir_pass.fold_constant(relay_prog)

# Print the transformed model
print(relay_prog)

In [ ]:
# Graph runtime
m = None

# Load pre-configured AutoTVM schedules
with autotvm.tophub.context(target):
    
    # Compile Relay program with AlterOpLayout disabled
    with relay.build_config(opt_level=3, disabled_pass={"AlterOpLayout"}):
        if target.device_name != "vta":
            graph, lib, params = relay.build(
                relay_prog, target=target,
                params=params, target_host=env.target_host)
        else:
            with vta.build_config():
                graph, lib, params = relay.build(
                    relay_prog, target=target,
                    params=params, target_host=env.target_host)

    # Send the inference library over to the remote RPC server
    temp = util.tempdir()
    lib.save(temp.relpath("graphlib.o"))
    remote.upload(temp.relpath("graphlib.o"))
    lib = remote.load_module("graphlib.o")

    # Graph runtime
    if env.TARGET == "sim":
        m = graph_runtime.create(graph, lib, ctx)
    elif env.TARGET == "pynq":
        # In hardware we'll use the debug runtime
        from tvm.contrib.debugger import debug_runtime
        m = debug_runtime.create(graph, lib, ctx)

Perform ResNet-18 inference
---------------------------
We run classification on an image sample from ImageNet
We just need to download the categories files, `synset.txt`
and an input test image.



In [ ]:
# Download ImageNet categories
categ_url = "https://github.com/uwsaml/web-data/raw/master/vta/models/"
categ_fn = "synset.txt"
download.download(join(categ_url, categ_fn), categ_fn)
synset = eval(open(categ_fn).read())

# Download test image
image_url = 'https://homes.cs.washington.edu/~moreau/media/vta/cat.jpg'
response = requests.get(image_url)

# Prepare test image for inference
image = Image.open(BytesIO(response.content)).resize((224, 224))
plt.imshow(image)
plt.show()
image = np.array(image) - np.array([123., 117., 104.])
image /= np.array([58.395, 57.12, 57.375])
image = image.transpose((2, 0, 1))
image = image[np.newaxis, :]
image = np.repeat(image, env.BATCH, axis=0)

# Set the network parameters and inputs
m.set_input(**params)
m.set_input('data', image)

# Perform inference: we run the module 4 times,
# and repeat 3 times to get error bounds
timer = m.module.time_evaluator("run", ctx, number=4, repeat=3)
tcost = timer()

# Get classification results
tvm_output = m.get_output(0, tvm.nd.empty((env.BATCH, 1000), "float32", remote.cpu(0)))
top_categories = np.argsort(tvm_output.asnumpy()[0])

# Report top-5 classification results
std = np.std(tcost.results) * 1000 / env.BATCH
mean = tcost.mean * 1000 / env.BATCH
print("%s prediction" % model)
print("                     #1:", synset[top_categories[-1]])
print("                     #2:", synset[top_categories[-2]])
print("                     #3:", synset[top_categories[-3]])
print("                     #4:", synset[top_categories[-4]])
print("                     #5:", synset[top_categories[-5]])
print("Performed inference in %.2fms/sample (std = %.2f)" % (mean, std))


In [ ]:
# Invoke the debug runtime to get a per-operator runtime breakdown
if env.TARGET == "pynq":
    m.run()